In [ ]:
import numpy as np
import math

In [ ]:
G = 6.6732e-11
t_step = 1000

In [ ]:
class Planets():
    def __init__(self, mass, x, v):
        self.m = mass
        self.x = x 
        self.v = v
        self.a = np.zeros((self.x.shape[0], 3), dtype=float)
        self.f = np.zeros((self.x.shape[0], 3), dtype=float)

    def forward(self, t_step):
        self.a = self.f / self.m
        self.v += self.a * t_step
        self.x += self.v * t_step

    def cal_force(self):
        self.f = np.zeros((self.x.shape[0], 3), dtype=float)
        for i in range(self.x.shape[0]):
            for j in range(self.x.shape[0]):
                if i == j: continue
                R = self.x[j] - self.x[i]
                #print(R.shape) (3,)
                R_mag = math.sqrt((self.x[i][0]-self.x[j][0])**2 + (self.x[i][1]-self.x[j][1])**2 + (self.x[i][2]-self.x[j][2])**2)
                self.f[i] += G * self.m[i][0] * self.m[j][0] / (R_mag*R_mag*R_mag) * R

    def update(self,t_step):
        self.cal_force()
        self.forward(t_step)

In [ ]:
from manim import *
import numpy as np

class PlanetarySimulation(ThreeDScene):
    def construct(self):
        # Set up the camera for 3D view
        self.set_camera_orientation(phi=60 * DEGREES, theta=-45 * DEGREES)

        """
        #sun_earth_mars example, sun put in front
        mass = np.array([1.989e30, 5.972e24, 6.39e23], dtype=float)  #mass for each planet
        pos = np.array([[0,0,0],                            
                       [1.496e11,0,0],
                       [-2.279e11,0,0]], dtype=float)                #displacement in x[x,y,z] for each planet
        vel = np.array([[0, 0, 0],
                       [0, 29780, 0],
                       [0, -24077, 0]], dtype=float)                 #velocity in v[x,y,z] for each planet

        spheres = []
        colors = [YELLOW, BLUE, RED]  
        radii = [0.5, 0.2, 0.15]
        """

        mass = np.array([
            [1.989e30],       # Sun
            [3.301e23],       # Mercury
            [4.867e24],       # Venus
            [5.972e24],       # Earth
            [7.348e22],       # Moon
            [6.417e23],       # Mars
            [1.898e27],       # Jupiter
            [8.932e22],       # Io
            [4.8e22],         # Europa
            [5.683e26],       # Saturn
            [1.345e23]        # Titan
        ], dtype=float)
        
        pos = np.array([
            [0, 0, 0],                          # Sun
            [5.791e10, 0, 0],                   # Mercury
            [1.082e11, 0, 0],                   # Venus
            [-1.496e11, 0, 0],                   # Earth
            [-1.496e11 + 3.844e8, 0, 0],         # Moon (relative to Earth)
            [2.279e11, 0, 0],                   # Mars
            [-7.785e11, 0, 0],                   # Jupiter
            [-7.785e11 + 4.22e8, 0, 0],          # Io (relative to Jupiter)
            [7.785e11 + 6.71e8, 0, 0],          # Europa (relative to Jupiter)
            [-1.434e12, 0, 0],                   # Saturn
            [-1.434e12 + 1.22e9, 0, 0]           # Titan (relative to Saturn)
        ], dtype=float)
        
        vel = np.array([
            [0, 0, 0],                          # Sun
            [0, 47360, 0],                      # Mercury
            [0, 35020, 0],                      # Venus
            [0, -29780, 0],                      # Earth
            [0, -29780 + 1022, 0],               # Moon (relative to Earth)
            [0, 24070, 0],                      # Mars
            [0, -13070, 0],                      # Jupiter
            [0, -13070 + 17330, 0],              # Io (relative to Jupiter)
            [0, 13070 + 13740, 0],              # Europa (relative to Jupiter)
            [0, -9690, 0],                       # Saturn
            [0, -9690 + 5570, 0]                 # Titan (relative to Saturn)
        ], dtype=float)

        spheres = []
        radii = [
            0.2,      # Sun
            0.05,     # Mercury
            0.08,      # Venus
            0.07,      # Earth
            0.02,     # Moon
            0.05,     # Mars
            0.13,      # Jupiter
            0.03,     # Io
            0.025,    # Europa
            0.15,     # Saturn
            0.04      # Titan
        ]
        colors = [
            YELLOW,   # Sun
            GRAY,     # Mercury
            ORANGE,   # Venus
            BLUE,     # Earth
            WHITE,    # Moon
            RED,      # Mars
            LIGHT_BROWN,  # Jupiter
            LIGHT_GRAY,   # Io
            LIGHT_GRAY,   # Europa
            GOLD,     # Saturn
            LIGHT_GRAY    # Titan
        ]

        planets = Planets(mass, pos, vel)

              
        for i in range(planets.x.shape[0]):
            sphere = Sphere(radius=radii[i]).move_to(planets.x[i] / 2e11)
            sphere.set_fill(color=colors[i], opacity=1.0)  # Set flat color
            spheres.append(sphere)

        self.add(*spheres)
        
        # Simulation loop
        
        for step in range(1200):#5256
            for i in range(53):
                planets.update(t_step)
            
            # Update sphere positions in the animation
            for i, sphere in enumerate(spheres):
                sphere.move_to(planets.x[i] / 2e11)

            self.renderer.camera.light_source.move_to(planets.x[0] / 2e11)
            """
            new_phi = 75 * DEGREES + (step / 10) * 10 * DEGREES  # Gradually change phi
            new_theta = -45 * DEGREES + (step / 10) * 20 * DEGREES  # Gradually change theta
            """
            # Animate the movement
            self.play(
                *[sphere.animate.move_to(planets.x[i] / 2e11) for i, sphere in enumerate(spheres)],
                #self.camera.animate.set_phi(new_phi),
                #self.camera.animate.set_theta(new_theta),
                run_time=0.03,
                rate_func=linear
            )
            """
            if step % 20 == 0:  # Move the camera every 20 steps
                new_phi = 75 * DEGREES + (step / 20) * 10 * DEGREES  # Gradually change phi
                new_theta = -45 * DEGREES + (step / 20) * 20 * DEGREES  # Gradually change theta
                self.move_camera(
                phi=new_phi,
                theta=new_theta,
                run_time=0.05,  # Match the run_time of the planet animation
                rate_func=linear
                )
            """
config["output_file"] = "Planetary_Simulation4.41.mp4"

# Run the simulation
if __name__ == "__main__":
    scene = PlanetarySimulation()
    scene.render()